## Function Examples
Below are some C# implementations of various ways that function may appear in a code base

#### Typical and Lambda Functions

In [ ]:
// typical function
public static int MultiplyBy2AndAdd(int x, int y) {
    return (x * 2) + (y * 2);
}

// lambda function
var IsMod3 = (int x) => x % 3 == 0; // aka: Func<int, bool> 

Console.WriteLine($"MultiplyBy2AndAdd(2, 3): {MultiplyBy2AndAdd(2, 3)}"); // == 10
Console.WriteLine($"IsMod(9): {IsMod3(9)}"); // == true


#### Delegate

Using the `Comparison<T>` delegate type from the `System` namespace
```csharp
public namespace System
{
    public delegate Comparison<T>(T left, T right);
}
```

In [ ]:
Comparison<string> sortAlphabetically = (left, right) =>
    left.CompareTo(right);

var beerList = new List<string> { "tripel", "ipa", "sour ale", "dunkel", "lager", "aged", "stout" };
beerList.Sort(sortAlphabetically); 
beerList.Display();

#### Dictionaries

In [ ]:
var times2 = new Dictionary<int, int>
{
    { 0, 0 },
    { 1, 2 },
    { 2, 4 },
    { 3, 6 },
    { 4, 8 }
};

Console.WriteLine($"times2[3]: {times2[3]}");

### e pluribus unum: creating a higher order function

Below is sort of the canonical example that we probably use everyday using the `Where` method. `Where` takes a function that has a type `T` input and produces a `bool` (aka: `Func<T, bool>`).

In [ ]:
beerList.Where(beer => beer.StartsWith("s")).Display();

index,value
0,sour ale
1,stout


We have already defined a HOF that produces a function, it is called `IsMod3`. That function produces a function that takes an input of `int` and returns a `bool`. 

In [ ]:
Enumerable.Range(0, 30).Where(IsMod3).Display();

index,value
0,0
1,3
2,6
3,9
4,12
5,15
6,18
7,21
8,24
9,27


_Bonus_: You can get pretty crazy with passing functions and one aspect of more advanced FP is partial applications. This allows you to create functions interatively. 

In [ ]:
// we have a function that outputs some log message
var log = (string level, string message) => Console.WriteLine($"{level.ToUpper()}: {message}");
log("WARNING", "We did something bad but not terrible.");
log("Error", "THIS WAS REALLY BAD!");
log("debug", "Meh, just some random data.");

// that could get tedious, so lets add some generic log level functions
// we will rewrite the log function so it can be partially applied
var newlog = (string level) => (string message) =>  Console.WriteLine($"{level.ToUpper()}: {message}");

// specialized logging functions
var errorLog = newlog("error");
var debugLog = newlog("Debug");
var warningLog = newlog("WARNING");

// using the specialized functions
errorLog("THIS IS BAD!");
debugLog("Some random info.");
warningLog("Not terrible but you should know about it.")


ERROR: THIS WAS REALLY BAD!
DEBUG: Meh, just some random data.
ERROR: THIS IS BAD!
DEBUG: Some random info.


## Impure Function example

In [ ]:
// Calculates the size of fermentation vessel we need
// based on the amount of ingredients
public class Ferment
{
    private List<string> ingredients = new();
    private string equipmentNeeded = "5 gallon fermenter";

    public void AddIngredient(string ingredient)
    {
        ingredients.Add(ingredient);
        if(ingredient.Length > 10) {
            equipmentNeeded = "15 gallon fermenter";
        } else if (ingredient.Length > 5) {
            equipmentNeeded = "10 gallon fermenter";
        }
    }

    public string GetEquipmentNeeded() {
        return equipmentNeeded;
    }

    public List<string> GetIngredients() {
        return ingredients;
    }
}

## Reimplemented as a pure function

## Side effects and how to deal with them

## Parallelizing pure functions is 🔥

### Honesty is the best policy
Writing a function that doesn't lie to me or others about what it does. 

## Immutability in C# 10

## Immutability before C# 10

## Copy on update

### Bonus items

Removing `null` with `Option<T>`